In [ ]:
from transformers import (
AutoTokenizer,
AutoModelForCausalLM,
BitsAndBytesConfig,
pipeline
)

import torch

import os

from langchain.llms import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Weaviate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, format_document
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string

from operator import itemgetter

import gradio as gr

In [ ]:


#Loading the Mistral Model
model_name='mistralai/Mistral-7B-Instruct-v0.2'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

# Building a LLM text-generation pipeline
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1024,
    device_map = 'auto',
)

llm = HuggingFacePipeline(pipeline= text_generation_pipeline)


In [ ]:

hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)


In [ ]:
import weaviate

client = weaviate.Client(url=  'https://superteams-810p8edk.weaviate.network')

In [ ]:
client.schema.delete_all()

In [ ]:


vectorstore = Weaviate.from_documents(
    [], embedding=hf_embeddings,
    client = client ,
    by_text= False
)


In [ ]:

# Simulate some document processing delay
textsplitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)


In [ ]:


#template to get the Standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

    Chat History:
    {chat_history}
    Follow Up Input: {question}
    Standalone question:
"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

#Function to create the context from retrieved documents
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")
def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

#Creating the template for the final answer
template = """Answer the question based only on the following context:
    {context}

    Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)


# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | llm
    | StrOutputParser(),
}

# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | vectorstore.as_retriever(search_kwargs = {'k':10}),
    "question": lambda x: x["standalone_question"],
}

# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}

# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | llm,
    "docs": itemgetter("docs"),
}



In [ ]:
def add_pdfs_to_vectorstore(files):

    saved_files_count = 0
    for file_path in files:
        file_name = os.path.basename(file_path)  # Extract the filename from the full path
        if file_name.lower().endswith('.pdf'):  # Check if the file is a PDF
            saved_files_count += 1
            loader_temp = PyPDFLoader(file_path)
            docs_temp = loader_temp.load_and_split(text_splitter=textsplitter)
            vectorstore.add_documents(docs_temp)

        else:
            print(f"Skipping non-PDF file: {file_name}")

    return f"Added {saved_files_count} PDF file(s) to vectorstore/"

In [ ]:
add_pdfs_to_vectorstore(['Goddess - Plot.pdf'])

In [52]:
conversational_memory = ConversationBufferMemory(
        return_messages=True, output_key="answer", input_key="question"
    )

In [50]:
def clear_vectordb(chatbot, msg):
    client.schema.delete_all()
    conversational_memory.clear()
    chatbot = ""
    msg = ""
    return chatbot, msg

In [51]:
def answer_query(message, chat_history):
    loaded_memory = RunnablePassthrough.assign(
        chat_history=RunnableLambda(conversational_memory.load_memory_variables) | itemgetter("history"),
    )
    chain = loaded_memory | standalone_question | retrieved_documents | answer

    inputs = {"question": message}
    result = chain.invoke(inputs)

    chat_history.append((message, result["answer"]))
    conversational_memory.save_context(inputs, {"answer": result["answer"]})

    return "", chat_history

In [ ]:
llm = load_llm()

hf_embeddings = embeddings_model()

vectorstore = initialize_vectorstore()

standalone_question, retrieved_documents, answer = return_chain_elements()

textsplitter = textsplitter()

conversational_memory = conversational_memory()

In [53]:
with gr.Blocks() as demo:

   with gr.Row():
       upload_files = gr.File(label= "Upload pdf files only", file_count= 'multiple')
       success_msg = gr.Text(value="")

   chatbot = gr.Chatbot()
   msg = gr.Textbox(label= "Enter your query here")
   clear = gr.ClearButton([msg, chatbot], value= "Clear VectorDB")


   upload_files.upload(add_pdfs_to_vectorstore, upload_files, success_msg)
   msg.submit(answer_query, [msg, chatbot], [msg, chatbot])

demo.launch(server_name='0.0.0.0', share= True)


/opt/conda/envs/pytorch/lib/python3.10/site-packages/gradio/routes.py:945: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
/opt/conda/envs/pytorch/lib/python3.10/site-packages/fastapi/applications.py:4495: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  return self.router.on_event(event_type)


Running on local URL:  http://0.0.0.0:7860
Running on public URL: https://defe2053333ace9cec.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/opt/conda/envs/pytorch/lib/python3.10/site-packages/starlette/templating.py:172: DeprecationWarning: The `name` is not the first parameter anymore. The first parameter should be the `Request` instance.
Replace `TemplateResponse(name, {"request": request})` by `TemplateResponse(request, name)`.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/envs/pytorch/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperat